In [ ]:
import Utils.merge as mg
import model as md
import torch

model1 = md.RoBERTa_Classify("roberta-large", 3)
model1.load_state_dict(torch.load("_fine_tuned_1.model"))

model2 = md.RoBERTa_Classify("roberta-large", 3)
model2.load_state_dict(torch.load("_fine_tuned_2.model"))

model3 = md.RoBERTa_Classify("roberta-large", 3)
model3.load_state_dict(torch.load("_fine_tuned_3.model"))

model4 = md.RoBERTa_Classify("roberta-large", 3)
model4.load_state_dict(torch.load("_fine_tuned_4.model"))

model_list = [model1, model2, model3, model4]

In [2]:
merged_model = mg.model_merge(model_list)

In [3]:
torch.save(merged_model.state_dict(), "_merged.model")

In [ ]:
#-------------------------------------------------------------#

In [1]:
import pandas as pd
import Utils.dataset_processing as dp
from transformers import AutoTokenizer
from torch.utils.data import random_split
import robust_loss
import model as md
import torch
import Utils.train as tr
import torch.nn as nn

merged_model = md.RoBERTa_Classify("roberta-large", 3)
merged_model.load_state_dict(torch.load("_merged.model"))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [2]:
model_stad = merged_model.state_dict()
for name, para in merged_model.named_parameters(): 
    model_stad[name][:] += (torch.rand(para.size())-0.5) * 0.05 * torch.std(para)
merged_model.load_state_dict(model_stad)
merged_model.cuda()

RoBERTa_Classify(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (La

In [3]:
merged_model.cuda()

RoBERTa_Classify(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (La

In [4]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large")
test_data_path = 'Data/test_data.jsonl'
test_datatable = dp.read_json_to_table(test_data_path, False)
test_dataloader = dp.dataset_loader(test_datatable, tokenizer)

In [5]:
labeled_datatable = list()

In [6]:
mapping = ['positive', 'negative', 'neutral']

for i in range(0, len(test_dataloader)):
    example = test_dataloader[i]
    predict = merged_model([example[0][0].view(1, example[0][0].shape[0]), example[0][1].view(1, example[0][1].shape[0])])
    predict = nn.Softmax()(predict).cpu()
    labeled_datatable.append([test_datatable[i][3], mapping[torch.argmax(predict[0]).detach().item()]])
    print(mapping[torch.argmax(predict[0]).detach().item()])
    print(torch.max(predict[0]).detach().item())

/home/s2320415/.local/lib/python3.11/site-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


neutral
0.9421783089637756
neutral
0.7585285305976868
positive
0.9373963475227356
negative
0.9428386688232422
negative
0.9525596499443054
neutral
0.9623499512672424
neutral
0.924670398235321
neutral
0.5786243677139282
neutral
0.8416203260421753
negative
0.6688303351402283
negative
0.9659677743911743
neutral
0.9495741128921509
negative
0.9489931464195251
neutral
0.9607998728752136
neutral
0.83112633228302
negative
0.8253676295280457
negative
0.9497100114822388
neutral
0.9511663317680359
negative
0.9005746841430664
negative
0.9413594007492065
neutral
0.9766358733177185
neutral
0.9476301670074463
positive
0.9191715717315674
neutral
0.4991227984428406
negative
0.9543262720108032
neutral
0.9100427031517029
negative
0.8972246050834656
neutral
0.8635624051094055
neutral
0.7944209575653076
neutral
0.9487084150314331
neutral
0.9774143695831299
positive
0.9513987898826599
negative
0.6138145327568054
neutral
0.9595370888710022
neutral
0.9260780215263367
negative
0.9050173759460449
neutral
0.94854

In [7]:
data = pd.DataFrame(data = labeled_datatable)
# PATH为导出文件的路径和文件名
data.to_csv('_res.csv')

In [9]:
import numpy as np
import pickle

seed = np.random.get_state()

with open("_step4_randomseed.pk", "wb") as file:
    pickle.dump(seed, file)

In [10]:
torch.save(merged_model.state_dict(), "_final.model")